# Fetch Annotations from Label-Studio and output to match YOLOv4 requirements

In [1]:
import os, json, copy
import pandas as pd
import numpy as np
from IPython.display import display
import commonsettings
from sklearn.model_selection import train_test_split
from functools import reduce

rawSrcAnnotations = commonsettings.getLabelStudioData()
baseConfigFilenames = commonsettings.generatePaths(specificProjName = '3classbeans_yolo4tiny_filtered_w512h512_20210711')
sourceRawSrcAnnotations = commonsettings.rawSrcAnnotationsToPdDF(rawSrcAnnotations)
#bad-IMAG0723.jpg

In [2]:
rawSrcAnnotations = copy.deepcopy(sourceRawSrcAnnotations)
origAllLabelKinds = reduce(lambda x,y: x.union(set(y)) if isinstance(x,set) else set(x).union(set(y)), rawSrcAnnotations['value.rectanglelabels'].tolist())
origAllLabelKinds = {v:key for key,v in enumerate(origAllLabelKinds)}

multiclass = True
if multiclass==False:
    rawSrcAnnotations['value.rectanglelabels'] = rawSrcAnnotations['value.rectanglelabels'].apply(lambda x: [x[0]])
    convertLabelDef1 = {key:'bean' for key,v in origAllLabelKinds.items()}
    convertLabelDef2 = {'bean':0}
else:
    convertLabelDef1 = {key:'Bad' for key in ['Pureblack','UnlabelledBadBean','Withered','Shells','MottledSpotted','Broken','Sour','Immature','Hulls','Unhulled','Oldpast','Moldy','Insect','ForeignMatter','Dead']}
    convertLabelDef1 = {**convertLabelDef1, **{key:key for key in ['UnlabelledBean','Good']}}
    convertLabelDef2 = {'Good':0,'UnlabelledBean':1,'Bad':2}

def covertLabelInputToOnlyOneClass(origLabels, multiclass=multiclass):
    first_try_convert = list(set([convertLabelDef2[convertLabelDef1[origLabel]] for origLabel in origLabels]))
    if multiclass==False:
        return first_try_convert
    else:
        for converted_label_kind in [convertLabelDef2['Good'],convertLabelDef2['Bad'],convertLabelDef2['UnlabelledBean']]:
            #print("checking if converted_label_kind {} in first_try_convert {}".format(converted_label_kind,first_try_convert))
            if converted_label_kind in first_try_convert:
                finalreturn =  converted_label_kind
                #print("found finalreturn {}".format(finalreturn))
                return [finalreturn]

rawSrcAnnotations_beanlabel = [covertLabelInputToOnlyOneClass(x) for x in rawSrcAnnotations['value.rectanglelabels'].tolist()] #rawSrcAnnotations['value.rectanglelabels'].apply(covertLabelInputToOnlyOneClass).astype(str)
rawSrcAnnotations['beanlabel'] = pd.Series(rawSrcAnnotations_beanlabel)

In [3]:
allTextFiles = {}
rawSrcAnnotations = rawSrcAnnotations[rawSrcAnnotations['created_username'].str.contains("tingjhenjiang")].reset_index(drop=True)

targetImageFilePaths = [
    {
        'targetImageFilePath': os.path.join(baseConfigFilenames['traindataSrcPathImgsBase'], f),
        'targetImageFilePathDisplay': os.path.join(baseConfigFilenames['traindataSrcPathImgsBaseDisplayed'], f).replace(os.sep,"/")
    }
    for f in rawSrcAnnotations['data.image.basename'].unique().tolist()
    ]
for f in rawSrcAnnotations['data.image.basename'].unique().tolist():
    toWriteTextFilePath = os.path.join(baseConfigFilenames['traindataSrcPathImgsBase'], f).replace(".jpg",".txt").replace(".JPG",".txt")
    toDisplayTextFilePath = os.path.join(baseConfigFilenames['traindataSrcPathImgsBaseDisplayed'], f)
    tempSrcDF = rawSrcAnnotations[rawSrcAnnotations['data.image.basename']==f].reset_index(drop=True)
    normalizedTempSrcDF = []
    for tempSrcDF_rown in range(len(tempSrcDF.index)):
        for beanlabel in tempSrcDF['beanlabel'][tempSrcDF_rown]:
            normalizedTempSrcDF.append(tempSrcDF.iloc[[tempSrcDF_rown],:].assign(beanlabel=beanlabel))
            #print(beanlabel)
    normalizedTempSrcDF = pd.concat(normalizedTempSrcDF).reset_index(drop=True)
    normalizedTempSrcDF['beanlabel'] = normalizedTempSrcDF['beanlabel'].astype(str)

    #<x_center> <y_center> <width> <height>
    normalizedTempSrcDF['annotationTxtContent'] = normalizedTempSrcDF.loc[:,['beanlabel','value.yolo.x.center','value.yolo.y.center','value.yolo.width','value.yolo.height']].values.tolist()
    normalizedTempSrcDF['annotationTxtContent'] = normalizedTempSrcDF['annotationTxtContent'].apply(lambda x: " ".join( map(str, x) ) )
    annotationTxtContent = "\n".join(normalizedTempSrcDF['annotationTxtContent'].tolist())
    allTextFiles[toWriteTextFilePath] = annotationTxtContent
    #if toWriteTextFilePath=='/opt/mnt/code-391ff5ac-6576-460f-ba4d-7e03433c68b6/Users/Jerry.Hsieh/coffeebeans/darknet/data/obj/good-IMAG0509.txt':
    #    break

In [ ]:
seed = 24
trainsetImagePaths, validationsetImagePaths = train_test_split(targetImageFilePaths,test_size=0.2,random_state=seed)
#trainsetImagePaths, validationsetImagePaths = train_test_split(trainsetImagePaths,test_size=0.2)

with open(baseConfigFilenames['allTakenImagesFilelist']['orig'], 'w+') as fp:
    fp.write("\n".join(baseConfigFilenames['allTakenImages']['display']))
    fp.close()
with open(baseConfigFilenames['trainListFileName']['orig'], 'w+') as fp:
    trainsetImagePaths = [p['targetImageFilePathDisplay'] for p in trainsetImagePaths]
    fp.write("\n".join(trainsetImagePaths))
    fp.close()
with open(baseConfigFilenames['validationListFileName']['orig'], 'w+') as fp:
    validationsetImagePaths = [p['targetImageFilePathDisplay'] for p in validationsetImagePaths]
    fp.write("\n".join(validationsetImagePaths))
    fp.close()
for textFilePath,nAnnotationsTxtContent in allTextFiles.items():
    #for label, labelkey in allLabelKinds.items():
    #    nAnnotationsTxtContent = nAnnotationsTxtContent.replace(label,str(labelkey))
    #display(nAnnotationsTxtContent)
    print("writing {}".format(textFilePath))
    f = open(textFilePath, "w+")
    f.write(nAnnotationsTxtContent)
    f.close()
with open(baseConfigFilenames['meaningJsonFileName']['orig'], 'w') as fp:
    json.dump(convertLabelDef2, fp)
with open(baseConfigFilenames['objNamesFileName']['orig'], 'w') as fp:
    fp.write("\n".join(convertLabelDef2.keys()))
    fp.close()
with open(baseConfigFilenames['objDataFileName']['orig'], 'w') as fp:
    objDataContent = """
        classes= {}
        train  = {}
        valid  = {}
        names = {}
        backup = {}
        """.format(
            len(convertLabelDef2.keys()),
            baseConfigFilenames['trainListFileName']['display'],
            baseConfigFilenames['validationListFileName']['display'],
            baseConfigFilenames['objNamesFileName']['display'],
            baseConfigFilenames['backupFileName']['display']
    )
    objDataContent = "\n".join([s.strip() for s in objDataContent.split("\n")]).strip()
    fp.write(objDataContent)
    fp.close()

# Move Images that have no annotations

In [2]:
import commonsettings
import os,re

In [ ]:
storeImagesPath = os.path.join(commonsettings.privateSettings['traindataSrcPathBaseDisplayed'], 'obj')
storeImagesPath_backupobj = storeImagesPath.replace('obj','backupobj')
allunder_storeImagesPath_backupobj = os.path.join(storeImagesPath_backupobj,'*')
newImagesPath = storeImagesPath.replace('obj','backupobj')
allFiles = os.listdir(storeImagesPath)
alljpgs = [f for f in allFiles if re.search(".jpg|.JPG",f)!=None]
onlyimgs = [f for f in alljpgs if (f.replace(".jpg",".txt").replace(".JPG",".txt") not in allFiles)]
for f in onlyimgs:
    #res = os.rename( os.path.join(storeImagesPath,f), os.path.join(newImagesPath,f))
    #print("Result of moving {} is {}".format(f,res))
    pass

In [17]:
compareImagefiles = ["mixed-IMAG1082.jpg",
"bad-IMAG0605.jpg",
"bad-IMAG0606.jpg",
"bad-IMAG0607.jpg",
"bad-IMAG0608.jpg",
"bad-IMAG0609.jpg",
"bad-IMAG0610.jpg",
"bad-IMAG0611.jpg",
"bad-IMAG0612.jpg",
"bad-IMAG0613.jpg",
"bad-IMAG0614.jpg",
"bad-IMAG0615.jpg",
"bad-IMAG0616.jpg",
"bad-IMAG0617.jpg",
"bad-IMAG0618.jpg",
"bad-IMAG0619.jpg",
"bad-IMAG0620.jpg",
"bad-IMAG0621.jpg",
"bad-IMAG0622.jpg",
"bad-IMAG0623.jpg",
"bad-IMAG0624.jpg",
"bad-IMAG0625.jpg",
"bad-IMAG0626.jpg",
"bad-IMAG0627.jpg",
"bad-IMAG0628.jpg",
"bad-IMAG0629.jpg",
"bad-IMAG0630.jpg",
"bad-IMAG0631.jpg",
"bad-IMAG0632.jpg",
"bad-IMAG0633.jpg",
"bad-IMAG0634.jpg",
"bad-IMAG0636.jpg",
"bad-IMAG0637.jpg",
"bad-IMAG0638.jpg",
"bad-IMAG0639.jpg",
"bad-IMAG0640.jpg",
"bad-IMAG0641.jpg",
"bad-IMAG0642.jpg",
"bad-IMAG0643.jpg",
"bad-IMAG0644.jpg",
"bad-IMAG0645.jpg",
"bad-IMAG0646.jpg",
"bad-IMAG0647.jpg",
"bad-IMAG0648.jpg",
"bad-IMAG0649.jpg",
"bad-IMAG0650.jpg",
"bad-IMAG0651.jpg",
"bad-IMAG0652.jpg",
"bad-IMAG0653.jpg",
"bad-IMAG0654.jpg",
"bad-IMAG0655.jpg",
"bad-IMAG0656.jpg",
"bad-IMAG0657.jpg",
"bad-IMAG0658.jpg",
"bad-IMAG0659.jpg",
"bad-IMAG0660.jpg",
"bad-IMAG0661.jpg",
"bad-IMAG0662.jpg",
"bad-IMAG0663.jpg",
"bad-IMAG0664.jpg",
"bad-IMAG0665.jpg",
"bad-IMAG0666.jpg",
"bad-IMAG0667.jpg",
"bad-IMAG0668.jpg",
"bad-IMAG0669.jpg",
"bad-IMAG0670.jpg",
"bad-IMAG0671.jpg",
"bad-IMAG0672.jpg",
"bad-IMAG0673.jpg",
"bad-IMAG0674.jpg",
"bad-IMAG0675.jpg",
"bad-IMAG0676.jpg",
"bad-IMAG0677.jpg",
"bad-IMAG0679.jpg",
"bad-IMAG0680.jpg",
"bad-IMAG0681.jpg",
"bad-IMAG0682.jpg",
"bad-IMAG0683.jpg",
"bad-IMAG0684.jpg",
"bad-IMAG0685.jpg",
"bad-IMAG0686.jpg",
"bad-IMAG0687.jpg",
"bad-IMAG0688.jpg",
"bad-IMAG0689.jpg",
"bad-IMAG0690.jpg",
"bad-IMAG0691.jpg",
"bad-IMAG0692.jpg",
"bad-IMAG0693.jpg",
"bad-IMAG0694.jpg",
"bad-IMAG0695.jpg",
"bad-IMAG0696.jpg",
"bad-IMAG0697.jpg",
"bad-IMAG0698.jpg",
"bad-IMAG0699.jpg",
"bad-IMAG0700.jpg",
"bad-IMAG0701.jpg",
"bad-IMAG0702.jpg",
"bad-IMAG0703.jpg",
"bad-IMAG0704.jpg",
"bad-IMAG0705.jpg",
"bad-IMAG0706.jpg",
"bad-IMAG0707.jpg",
"bad-IMAG0708.jpg",
"bad-IMAG0709.jpg",
"bad-IMAG0710.jpg",
"bad-IMAG0710_.jpg",
"bad-IMAG0711.jpg",
"bad-IMAG0712.jpg",
"bad-IMAG0713.jpg",
"bad-IMAG0714.jpg",
"bad-IMAG0715.jpg",
"bad-IMAG0716.jpg",
"bad-IMAG0717.jpg",
"bad-IMAG0718.jpg",
"bad-IMAG0720.jpg",
"bad-IMAG0721.jpg",
"bad-IMAG0722.jpg",
"bad-IMAG0723.jpg",
"bad-IMAG0724.jpg",
"bad-IMAG0725.jpg",
"bad-IMAG0726.jpg",
"bad-IMAG0727.jpg",
"bad-IMAG0728.jpg",
"bad-IMAG0729.jpg",
"bad-IMAG0730.jpg",
"bad-IMAG0731.jpg",
"bad-IMAG0732.jpg",
"bad-IMAG0733.jpg",
"bad-IMAG0734.jpg",
"bad-IMAG0735.jpg",
"bad-IMAG0736.jpg",
"bad-IMAG0737.jpg",
"bad-IMAG0738.jpg",
"bad-IMAG0739.jpg",
"bad-IMAG0740.jpg",
"bad-IMAG0741.jpg",
"bad-IMAG0742.jpg",
"bad-IMAG0743.jpg",
"bad-IMAG0744.jpg",
"bad-IMAG0745.jpg",
"bad-IMAG0746.jpg",
"bad-IMAG0748.jpg",
"bad-IMAG0749.jpg",
"bad-IMAG0750.jpg",
"bad-IMAG0751.jpg",
"bad-IMAG0752.jpg",
"bad-IMAG0753.jpg",
"bad-IMAG0754.jpg",
"bad-IMAG0755.jpg",
"bad-IMAG0756.jpg",
"bad-IMAG0757.jpg",
"bad-IMAG0758.jpg",
"bad-IMAG0759.jpg",
"bad-IMAG0760.jpg",
"bad-IMAG0761.jpg",
"bad-IMAG0762.jpg",
"bad-IMAG0763.jpg",
"bad-IMAG0764.jpg",
"bad-IMAG0765.jpg",
"bad-IMAG0766.jpg",
"bad-IMAG0767.jpg",
"bad-IMAG0768.jpg",
"bad-IMAG0769.jpg",
"bad-IMAG0770.jpg",
"bad-IMAG0771.jpg",
"bad-IMAG0772.jpg",
"bad-IMAG0773.jpg",
"bad-IMAG0774.jpg",
"bad-IMAG0775.jpg",
"bad-IMAG0776.jpg",
"bad-IMAG0777.jpg",
"bad-IMAG0778.jpg",
"bad-IMAG0779.jpg",
"bad-IMAG0780.jpg",
"bad-IMAG0781.jpg",
"bad-IMAG0782.jpg",
"good-20210420_153827.jpg",
"good-20210420_153832.jpg",
"good-20210420_153836.jpg",
"good-20210420_153855.jpg",
"good-20210420_153900.jpg",
"good-20210420_153905.jpg",
"good-20210420_153912.jpg",
"good-20210420_153919.jpg",
"good-20210420_153923.jpg",
"good-20210420_153930.jpg",
"good-20210420_154046.jpg",
"good-20210420_154053.jpg",
"good-20210420_154100.jpg",
"good-20210420_154128.jpg",
"good-20210420_154150.jpg",
"good-20210420_154204.jpg",
"good-20210420_154216.jpg",
"good-20210420_154348.jpg",
"good-20210420_154356.jpg",
"good-20210420_154405.jpg",
"good-20210420_154413.jpg",
"good-20210420_154423.jpg",
"good-20210420_154436.jpg",
"good-20210420_154457.jpg",
"good-20210420_154505.jpg",
"good-20210420_154512.jpg",
"good-20210420_154526.jpg",
"good-20210420_154645.jpg",
"good-20210420_154709.jpg",
"good-20210420_154719.jpg",
"good-20210420_154730.jpg",
"good-20210420_154746.jpg",
"good-20210420_154806.jpg",
"good-20210420_154830.jpg",
"good-20210420_154952.jpg",
"good-20210420_155001.jpg",
"good-20210420_155008.jpg",
"good-20210420_155018.jpg",
"good-20210420_155028.jpg",
"good-20210420_155034.jpg",
"good-20210420_155048.jpg",
"good-20210420_155056.jpg",
"good-20210420_155104.jpg",
"good-20210420_155109.jpg",
"good-20210420_155219.jpg",
"good-20210420_155246.jpg",
"good-20210420_155258.jpg",
"good-20210420_155304.jpg",
"good-20210420_155310.jpg",
"good-20210420_155317.jpg",
"good-20210420_155323.jpg",
"good-20210420_155328.jpg",
"good-20210420_155336.jpg",
"good-20210420_155343.jpg",
"good-20210420_155352.jpg",
"good-IMAG0488.jpg",
"good-IMAG0489.jpg",
"good-IMAG0490.jpg",
"good-IMAG0491.jpg",
"good-IMAG0492.jpg",
"good-IMAG0493.jpg",
"good-IMAG0494.jpg",
"good-IMAG0495.jpg",
"good-IMAG0496.jpg",
"good-IMAG0497.jpg",
"good-IMAG0498.jpg",
"good-IMAG0499.jpg",
"good-IMAG0500.jpg",
"good-IMAG0501.jpg",
"good-IMAG0502.jpg",
"good-IMAG0503.jpg",
"good-IMAG0504.jpg",
"good-IMAG0505.jpg",
"good-IMAG0506.jpg",
"good-IMAG0507.jpg",
"good-IMAG0508.jpg",
"good-IMAG0509.jpg",
"good-IMAG0510.jpg",
"good-IMAG0511.jpg",
"good-IMAG0512.jpg",
"good-IMAG0514.jpg",
"good-IMAG0515.jpg",
"good-IMAG0523.jpg",
"good-IMAG0531.jpg",
"good-IMAG0533.jpg",
"good-IMAG0534.jpg",
"good-IMAG0535.jpg",
"good-IMAG0536.jpg",
"good-IMAG0538.jpg",
"good-IMAG0543.jpg",
"good-IMAG0544.jpg",
"good-IMAG0545.jpg",
"good-IMAG0546.jpg",
"good-IMAG0547.jpg",
"good-IMAG0548.jpg",
"good-IMAG0549.jpg",
"good-IMAG0550.jpg",
"good-IMAG0551.jpg",
"good-IMAG0552.jpg",
"good-IMAG0553.jpg",
"good-IMAG0554.jpg",
"good-IMAG0559.jpg",
"good-IMAG0560.jpg",
"good-IMAG0561.jpg",
"good-IMAG0562.jpg",
"good-IMAG0563.jpg",
"good-IMAG0564.jpg",
"good-IMAG0565.jpg",
"good-IMAG0566.jpg",
"good-IMAG0567.jpg",
"good-IMAG0568.jpg",
"good-IMAG0569.jpg",
"good-IMAG0570.jpg",
"good-IMAG0571.jpg",
"good-IMAG0572.jpg",
"good-IMAG0573.jpg",
"good-IMAG0574.jpg",
"good-IMAG0575.jpg",
"good-IMAG0576.jpg",
"good-IMAG0577.jpg",
"good-IMAG0578.jpg",
"good-IMAG0579.jpg",
"good-IMAG0580.jpg",
"good-IMAG0581.jpg",
"good-IMAG0582.jpg",
"good-IMAG0583.jpg",
"good-IMAG0584.jpg",
"good-IMAG0585.jpg",
"good-IMAG0586.jpg",
"good-IMAG0587.jpg",
"good-IMAG0588.jpg",
"good-IMAG0589.jpg",
"good-IMAG0590.jpg",
"good-IMAG0591.jpg",
"good-IMAG0592.jpg",
"good-IMAG0593.jpg",
"good-IMAG0594.jpg",
"good-IMAG0595.jpg",
"good-IMAG0597.jpg",
"good-IMAG0598.jpg",
"good-IMAG0599.jpg",
"good-IMAG0600.jpg",
"good-IMAG0601.jpg",
"good-IMAG0602.jpg",
"good-IMAG0603.jpg",
"good-IMAG0797.jpg",
"good-IMAG0798.jpg",
"good-IMAG0799.jpg",
"good-IMAG0800.jpg",
"good-IMAG0801.jpg",
"good-IMAG0802.jpg",
"good-IMAG0803.jpg",
"good-IMAG0804.jpg",
"good-IMAG0805.jpg",
"good-IMAG0806.jpg",
"good-IMAG0807.jpg",
"good-IMAG0808.jpg",
"good-IMAG0809.jpg",
"good-IMAG0810.jpg",
"good-IMAG0811.jpg",
"good-IMAG0812.jpg",
"good-IMAG0813.jpg",
"good-IMAG0814.jpg",
"good-IMAG0815.jpg",
"good-IMAG0816.jpg",
"good-IMAG0817.jpg",
"good-IMAG0818.jpg",
"good-IMAG0819.jpg",
"good-IMAG0820.jpg",
"good-IMAG0821.jpg",
"good-IMAG0822.jpg",
"good-IMAG0823.jpg",
"good-IMAG0824.jpg",
"good-IMAG0825.jpg",
"good-IMAG0826.jpg",
"good-IMAG0827.jpg",
"good-IMAG0828.jpg",
"good-IMAG0829.jpg",
"good-IMAG0830.jpg",
"good-IMAG0831.jpg",
"good-IMAG0832.jpg",
"good-IMAG0833.jpg",
"good-IMAG0834.jpg",
"good-IMAG0835.jpg",
"good-IMAG0836.jpg",
"good-IMAG0837.jpg",
"good-IMAG0838.jpg",
"good-IMAG0839.jpg",
"good-IMAG0840.jpg",
"good-IMAG0841.jpg",
"good-IMAG0842.jpg",
"good-IMAG0843.jpg",
"good-IMAG0844.jpg",
"good-IMAG0845.jpg",
"good-IMAG0846.jpg",
"good-IMAG0847.jpg",
"good-IMAG0848.jpg",
"good-IMAG0849.jpg",
"good-IMAG0850.jpg",
"good-IMAG0851.jpg",
"good-IMAG0852.jpg",
"good-IMAG0853.jpg",
"good-IMAG0854.jpg",
"good-IMAG0855.jpg",
"good-IMAG0856.jpg",
"good-IMAG0857.jpg",
"good-IMAG0858.jpg",
"good-IMAG0859.jpg",
"good-IMAG0860.jpg",
"good-IMAG0861.jpg",
"good-IMAG0862.jpg",
"good-IMAG0863.jpg",
"good-IMAG0864.jpg",
"good-IMAG0865.jpg",
"good-IMAG0866.jpg",
"good-IMAG0867.jpg",
"good-IMAG0868.jpg",
"good-IMAG0869.jpg",
"good-IMAG0870.jpg",
"good-IMAG0871.jpg",
"good-IMAG0872.jpg",
"good-IMAG0873.jpg",
"good-IMAG0874.jpg",
"good-IMAG0875.jpg",
"good-IMAG0876.jpg",
"good-IMAG0877.jpg",
"good-IMAG0878.jpg",
"good-IMAG0879.jpg",
"good-IMAG0880.jpg",
"good-IMAG0881.jpg",
"good-IMAG0882.jpg",
"good-IMAG0883.jpg",
"good-IMAG0884.jpg",
"good-IMAG0885.jpg",
"good-IMAG0886.jpg",
"good-IMAG0887.jpg",
"good-IMAG0888.jpg",
"good-IMAG0889.jpg",
"good-IMAG0890.jpg",
"good-IMAG0891.jpg",
"good-IMAG0892.jpg",
"good-IMAG0893.jpg",
"good-IMAG0894.jpg",
"good-IMAG0895.jpg",
"good-IMAG0896.jpg",
"good-IMAG0897.jpg",
"good-IMAG0898.jpg",
"good-IMAG0899.jpg",
"good-IMAG0900.jpg",
"good-IMAG0901.jpg",
"good-IMAG0902.jpg",
"good-IMAG0903.jpg",
"good-IMAG0904.jpg",
"good-IMAG0905.jpg",
"good-IMAG0906.jpg",
"good-IMAG0907.jpg",
"good-IMAG0908.jpg",
"good-IMAG0909.jpg",
"good-IMAG0910.jpg",
"good-IMAG0911.jpg",
"good-IMAG0912.jpg",
"good-IMAG0913.jpg",
"good-IMAG0914.jpg",
"good-IMAG0915.jpg",
"good-IMAG0916.jpg",
"good-IMAG0917.jpg",
"good-IMAG0918.jpg",
"good-IMAG0919.jpg",
"good-IMAG0920.jpg",
"good-IMAG0921.jpg",
"good-IMAG0922.jpg",
"good-IMAG0923.jpg",
"good-IMAG0924.jpg",
"good-IMAG0925.jpg",
"good-IMAG0926.jpg",
"good-IMAG0927.jpg",
"good-IMAG0928.jpg",
"good-IMAG0929.jpg",
"good-IMAG0930.jpg",
"good-IMAG0931.jpg",
"good-IMAG0932.jpg",
"good-IMAG0933.jpg",
"good-IMAG0934.jpg",
"good-IMAG0935.jpg",
"good-IMAG0936.jpg",
"good-IMAG0938.jpg",
"good-IMAG0939.jpg",
"good-IMAG0940.jpg",
"good-IMAG0941.jpg",
"good-IMAG0942.jpg",
"good-IMG_6402.JPG",
"good-IMG_6403.JPG",
"good-IMG_6404.JPG",
"good-IMG_6405.JPG",
"good-IMG_6406.JPG",
"good-IMG_6407.JPG",
"good-IMG_6408.JPG",
"good-IMG_6409.JPG",
"good-IMG_6410.JPG",
"good-IMG_6411.JPG",
"mixed-20210420_155504.jpg",
"mixed-20210420_155513.jpg",
"mixed-20210420_155528.jpg",
"mixed-20210420_155540.jpg",
"mixed-20210420_155556.jpg",
"mixed-20210420_155607.jpg",
"mixed-20210420_155613.jpg",
"mixed-20210420_155618.jpg",
"mixed-20210420_155638.jpg",
"mixed-20210420_155644.jpg",
"mixed-20210420_155649.jpg",
"mixed-20210420_155655.jpg",
"mixed-20210420_155700.jpg",
"mixed-20210420_155820.jpg",
"mixed-20210420_155828.jpg",
"mixed-20210420_155836.jpg",
"mixed-20210420_155843.jpg",
"mixed-20210420_155848.jpg",
"mixed-20210420_155854.jpg",
"mixed-20210420_155859.jpg",
"mixed-20210420_155916.jpg",
"mixed-20210420_155925.jpg",
"mixed-20210420_155935.jpg",
"mixed-20210420_155939.jpg",
"mixed-20210420_160049.jpg",
"mixed-20210420_160056.jpg",
"mixed-20210420_160102.jpg",
"mixed-20210420_160109.jpg",
"mixed-20210420_160115.jpg",
"mixed-20210420_160120.jpg",
"mixed-20210420_160135.jpg",
"mixed-20210420_160143.jpg",
"mixed-20210420_160149.jpg",
"mixed-20210420_160157.jpg",
"mixed-20210420_160208.jpg",
"mixed-20210420_160214.jpg",
"mixed-20210420_160221.jpg",
"mixed-20210420_160227.jpg",
"mixed-20210420_160235.jpg",
"mixed-IMAG0943.jpg",
"mixed-IMAG0944.jpg",
"mixed-IMAG0945.jpg",
"mixed-IMAG0946.jpg",
"mixed-IMAG0947.jpg",
"mixed-IMAG0948.jpg",
"mixed-IMAG0949.jpg",
"mixed-IMAG0950.jpg",
"mixed-IMAG0951.jpg",
"mixed-IMAG0952.jpg",
"mixed-IMAG0953.jpg",
"mixed-IMAG0954.jpg",
"mixed-IMAG0955.jpg",
"mixed-IMAG0956.jpg",
"mixed-IMAG0957.jpg",
"mixed-IMAG0958.jpg",
"mixed-IMAG0959.jpg",
"mixed-IMAG0960.jpg",
"mixed-IMAG0961.jpg",
"mixed-IMAG0962.jpg",
"mixed-IMAG0963.jpg",
"mixed-IMAG0964.jpg",
"mixed-IMAG0965.jpg",
"mixed-IMAG0966.jpg",
"mixed-IMAG0967.jpg",
"mixed-IMAG0968.jpg",
"mixed-IMAG0969.jpg",
"mixed-IMAG0970.jpg",
"mixed-IMAG0971.jpg",
"mixed-IMAG0972.jpg",
"mixed-IMAG0973.jpg",
"mixed-IMAG0974.jpg",
"mixed-IMAG0975.jpg",
"mixed-IMAG0976.jpg",
"mixed-IMAG0977.jpg",
"mixed-IMAG0978.jpg",
"mixed-IMAG0980.jpg",
"mixed-IMAG0981.jpg",
"mixed-IMAG0982.jpg",
"mixed-IMAG0983.jpg",
"mixed-IMAG0984.jpg",
"mixed-IMAG0985.jpg",
"mixed-IMAG0986.jpg",
"mixed-IMAG0987.jpg",
"mixed-IMAG0988.jpg",
"mixed-IMAG0989.jpg",
"mixed-IMAG0990.jpg",
"mixed-IMAG0991.jpg",
"mixed-IMAG0992.jpg",
"mixed-IMAG0993.jpg",
"mixed-IMAG0994.jpg",
"mixed-IMAG0995.jpg",
"mixed-IMAG0996.jpg",
"mixed-IMAG0997.jpg",
"mixed-IMAG0998.jpg",
"mixed-IMAG0999.jpg",
"mixed-IMAG1000.jpg",
"mixed-IMAG1001.jpg",
"mixed-IMAG1002.jpg",
"mixed-IMAG1003.jpg",
"mixed-IMAG1004.jpg",
"mixed-IMAG1005.jpg",
"mixed-IMAG1006.jpg",
"mixed-IMAG1007.jpg",
"mixed-IMAG1008.jpg",
"mixed-IMAG1009.jpg",
"mixed-IMAG1010.jpg",
"mixed-IMAG1011.jpg",
"mixed-IMAG1012.jpg",
"mixed-IMAG1013.jpg",
"mixed-IMAG1014.jpg",
"mixed-IMAG1015.jpg",
"mixed-IMAG1016.jpg",
"mixed-IMAG1017.jpg",
"mixed-IMAG1018.jpg",
"mixed-IMAG1019.jpg",
"mixed-IMAG1020.jpg",
"mixed-IMAG1021.jpg",
"mixed-IMAG1022.jpg",
"mixed-IMAG1023.jpg",
"mixed-IMAG1024.jpg",
"mixed-IMAG1025.jpg",
"mixed-IMAG1026.jpg",
"mixed-IMAG1027.jpg",
"mixed-IMAG1028.jpg",
"mixed-IMAG1029.jpg",
"mixed-IMAG1030.jpg",
"mixed-IMAG1031.jpg",
"mixed-IMAG1032.jpg",
"mixed-IMAG1033.jpg",
"mixed-IMAG1034.jpg",
"mixed-IMAG1035.jpg",
"mixed-IMAG1036.jpg",
"mixed-IMAG1037.jpg",
"mixed-IMAG1038.jpg",
"mixed-IMAG1039.jpg",
"mixed-IMAG1040.jpg",
"mixed-IMAG1041.jpg",
"mixed-IMAG1042.jpg",
"mixed-IMAG1043.jpg",
"mixed-IMAG1044.jpg",
"mixed-IMAG1045.jpg",
"mixed-IMAG1046.jpg",
"mixed-IMAG1047.jpg",
"mixed-IMAG1048.jpg",
"mixed-IMAG1049.jpg",
"mixed-IMAG1050.jpg",
"mixed-IMAG1051.jpg",
"mixed-IMAG1052.jpg",
"mixed-IMAG1053.jpg",
"mixed-IMAG1054.jpg",
"mixed-IMAG1055.jpg",
"mixed-IMAG1056.jpg",
"mixed-IMAG1057.jpg",
"mixed-IMAG1058.jpg",
"mixed-IMAG1059.jpg",
"mixed-IMAG1060.jpg",
"mixed-IMAG1061.jpg",
"mixed-IMAG1062.jpg",
"mixed-IMAG1063.jpg",
"mixed-IMAG1064.jpg",
"mixed-IMAG1065.jpg",
"mixed-IMAG1066.jpg",
"mixed-IMAG1067.jpg",
"mixed-IMAG1068.jpg",
"mixed-IMAG1069.jpg",
"mixed-IMAG1070.jpg",
"mixed-IMAG1071.jpg",
"mixed-IMAG1072.jpg",
"mixed-IMAG1073.jpg",
"mixed-IMAG1074.jpg",
"mixed-IMAG1075.jpg",
"mixed-IMAG1076.jpg",
"mixed-IMAG1077.jpg",
"mixed-IMAG1078.jpg",
"mixed-IMAG1079.jpg",
"mixed-IMAG1080.jpg",
"mixed-IMAG1081.jpg",]

set(compareImagefiles)-set(alljpgs)

{'bad-IMAG0710_.jpg'}

In [ ]:
!mv {allunder_storeImagesPath_backupobj} {storeImagesPath} -f